In [13]:
## Imports
import os
import sys
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, MaxPooling2D, GlobalMaxPool2D, Conv2D, Conv2DTranspose, concatenate, add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [14]:
from dataloaders.TGSSaltDataloader import *
from PIL import Image


In [15]:
# dataset: https://www.kaggle.com/c/tgs-salt-identification-challenge 
# TGSSalt
# file_list = ['datasets/TGSSalt/train/images', 'datasets/TGSSalt/train/masks', 'datasets/TGSSalt/test/images']
# test에는 labeling 안된 데이터만 있어서 사용하지 않고자 함.
file_list = ['datasets/TGSSalt/train/images', 'datasets/TGSSalt/train/masks']
x_train, y_train, x_test = list(), list(), list()

for i in range(len(file_list)):
    # print("file_list : ", i)
    for file_name in [file for file in os.listdir(file_list[i])]:
        # print("\tfile_name : ", file_name)
        if i == 0 or i == 2:
            x_data_ = np.array(Image.open(file_list[i] + "/" + file_name))
            if x_data_.shape[-1] == 4:    
                x_data_ = x_data_[:, :, :-1]
                
            if i == 0:
                x_train += [x_data_]
            else:
                x_test += [x_data_]
        else:
            y_data_ = np.array(Image.open(file_list[i] + "/" + file_name))
            y_train += [y_data_]
            
x_train, y_train, x_test = np.array(x_train), np.array(y_train), np.array(x_test) 

In [21]:
# Converting the image into the desired form
def load_image(img_path, show=False):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (128,128))
    img_tensor = img_to_array(img)                   
    #img_tensor = np.expand_dims(img_tensor, axis=0)         

    return img_tensor

x = []
y = []

def get_image(path):
    data =[]
    for subdir, dirs, files in os.walk(path):
        for f in files:
            path = os.path.join(subdir, f)
            img = load_image(path)     
            img = img/255.0
            data.append(img)
    return data


x = get_image('datasets/TGSSalt/train/images')
y = get_image('datasets/TGSSalt/train/masks')

x = np.asarray(x)
y = np.asarray(y)